In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
accom = pd.read_csv("/content/drive/MyDrive/yeoreodigm/data_files/09.15_accomodation.csv",index_col=0)

In [3]:
tmp = [i for i in range(1,923)]

In [4]:
accom['id'] = tmp

In [5]:
accom.columns

Index(['contentsid', 'title', 'roadaddress', 'tag', 'introduction', 'latitude',
       'longitude', 'contentscd.label', 'phoneno', 'repPhoto.photoid.imgpath',
       'repPhoto.photoid.thumbnailpath', '인기점수', 'tag_len', 'popular_tag',
       'len', 'id'],
      dtype='object')

In [6]:
accom = accom [['id','title', 'roadaddress','popular_tag']]
accom.head(10)

,id,title,roadaddress,popular_tag
3378,1,제주조천스위스마을게스트하우스,제주특별자치도 제주시 조천읍 함와로 566-27,"게스트하우스,숙소"
3801,2,해비치호텔앤드리조트,제주특별자치도 서귀포시 표선면 민속해안로 537,"호텔,숙소,양식레스토랑,가족호텔,주차장,수영장,리조트,공공와이파이존,관광호텔"
3976,3,나미송 머무는 곳,제주특별자치도 제주시 애월읍 평화로 2476,"민박,고향민박,정원,주방기구,자연경관,독채"
4240,4,신라호텔제주,제주도 서귀포시 중문관광로 72번길 75,"호텔,게스트하우스,숙소,양식레스토랑,주차장,수영장,공공와이파이존,관광호텔"
3162,5,롯데호텔 제주,제주특별자치도 서귀포시 중문관광로72번길 35,"호텔,숙소,조식,양식레스토랑,가족호텔,주차장,자연경관,수영장,리조트,공공와이파이존,..."
3169,6,대명리조트 제주,제주특별자치도 제주시 조천읍 신북로 577,"휴양콘도,숙소,조식,주차장,리조트"
4022,7,휘닉스 제주 섭지코지,제주특별자치도 서귀포시 성산읍 섭지코지로 107,"휴양콘도,양식레스토랑,가족,주차장,조식 포함,주방기구,수영장,교통,리조트,공공와이파이존"
3615,8,캠핑트리펜션,제주특별자치도 제주시 애월읍 광상로 115,"민박,숙소,펜션,휴양펜션,바비큐,수영장,독채"
3606,9,제주가좋아서 펜션,제주특별자치도 제주시 구좌읍 비자림로 1569,"숙소,안전인증민박,펜션,휴양펜션,바비큐,수영장,공공와이파이존"
3412,10,금호리조트제주,제주특별자치도 서귀포시 남원읍 태위로 522-12,"온돌방,휴양콘도,숙소,주차장,자연경관,리조트"


###숙박 상위 40개 태그 확인

In [7]:
tour = pd.read_csv("/content/drive/MyDrive/yeoreodigm/data_files/tourist_attraction_with_rating_sort_by_type.csv",index_col = 0)
accomodation = tour.loc[tour['contentscd.label']=='숙박']
accomodation_top40 = accomodation['tag'].str.get_dummies(sep=',').mean().sort_values(ascending=False)[:40].index
accomodation_top40

Index(['숙소', '공공와이파이존', '펜션', '휴양펜션', '주차장', '바비큐', '휴식', '해수욕장', '민박', '자연경관',
       '독채', '가족', '게스트하우스', '호텔', '해변', '온돌방', '단체여행객', '수영장', '농어촌민박',
       '조식 포함', '조식', '관광호텔', '체험', '리조트', '교통', '양식레스토랑', '정원', '수상레저', '카페',
       '주방기구', '휴양콘도', '힐링', '공항', '안전인증민박', '힐링쉼터', '가족호텔', '고향민박', '해안도로',
       '추자도', '물품보관서비스'],
      dtype='object')

In [8]:
accom_type = {'펜션/민박':['펜션','휴양펜션','민박','고향민박','안전인증민박','농어촌민박'],\
              '게스트하우스' : ['게스트하우스'], '호텔/콘도' : ['호텔','관광호텔','가족호텔','휴양콘도','리조트']}

In [9]:
accom_type = {'펜션':['펜션','휴양펜션','독채'],\
              '게스트하우스' : ['게스트하우스'], '호텔/콘도' : ['호텔','관광호텔','가족호텔','휴양콘도','리조트']}

###유명태그 없는 결측치 처리

In [10]:
accom.isnull().sum()

id             0
title          0
roadaddress    0
popular_tag    1
dtype: int64

In [11]:
accom[accom['popular_tag'].isnull()]

,id,title,roadaddress,popular_tag
1149,390,제주캠핑올레,제주도 서귀포시 표선면 토산망동로 15,NaN


In [12]:
accom['popular_tag'] = accom['popular_tag'].fillna('')

In [13]:
accom['popular_tag'] = accom['popular_tag'].apply(lambda x: x.split(','))

In [14]:
set([11]) & set(accom.sample()['popular_tag'].values[0])

set()

##중복 및 0개의 숙소타입을 가지는 값들 확인

In [15]:
check_list = []

for acc in accom.values:
  # print(acc)
  id,title,address, tag = acc[:]
  my_type = []
  
  #호텔/콘도/리조트, 민박, 펜션, 게스트하우스 이게 명칭으로 있는거는 확정짓기
  if '호텔' in title or '콘도' in title or '리조트' in title:
    # check_list.append('호텔/콘도')
    continue
  elif '펜션' in title:
    # check_list.append('펜션/민박')
    continue
  elif '게스트하우스' in title:
    # check_list.append('게스트하우스')
    continue
  elif '모텔' in title:
    # check_list.append('모텔')
    continue
  else:    
    for key in accom_type.keys():
      if set(accom_type[key]) & set(tag) != set(): # 교집합이 있다면
        # print(set(accom_type[key]) & set(tag))
        my_type.append(key)
    
  if len(my_type) != 1: #해당하는 타입이 하나도 없거나 중복된다면 체크리스트에 넣자
    check_list.append(id)

In [16]:
len(check_list)

141

In [17]:
check_list[:10]

[39, 54, 64, 65, 67, 80, 101, 116, 119, 187]

In [18]:
for id in check_list:
  now = accom[accom['id']==id].values[0]
  
  id,title = now[:2]
  tag = now[-1]
  print(id,'\t',title,'\t',tag)
  

39 	 틸다하우스 	 ['숙소', '휴식', '조식', '바비큐', '해수욕장', '공공와이파이존']
54 	 소풍(월정리) 	 ['숙소', '안전인증민박', '조식 포함', '해수욕장', '자연경관']
64 	 이디하우스n카페 	 ['게스트하우스', '숙소', '휴식', '조식', '주차장', '카페', '자연경관', '독채', '공공와이파이존']
65 	 비안 	 ['게스트하우스', '숙소', '휴식', '주차장', '조식 포함', '독채', '공공와이파이존']
67 	 마이테르유스호스텔 	 ['힐링쉼터', '수영장']
80 	 제주해나루 	 ['주차장', '숙소', '안전인증민박', '공공와이파이존']
101 	 팜핑제주 	 ['바비큐', '숙소', '공공와이파이존', '교통']
116 	 다락마마 	 ['숙소', '안전인증민박', '바비큐', '해수욕장', '공공와이파이존']
119 	 하도리보통날 	 ['민박', '숙소', '휴식', '고향민박', '주차장', '조식 포함', '공공와이파이존']
187 	 표선이레하우스 	 ['게스트하우스', '숙소', '휴식', '조식', '펜션', '휴양펜션', '카페', '해수욕장', '자연경관']
190 	 디오션힐 	 ['호텔', '펜션', '휴양펜션', '바비큐', '주차장', '해변', '자연경관', '해수욕장', '공공와이파이존']
206 	 비아 제주 	 ['민박', '숙소', '힐링', '주차장', '공공와이파이존']
227 	 유월그리고열두마루 	 ['민박', '숙소', '휴식', '조식', '농어촌민박', '공공와이파이존']
239 	 솔민박 	 ['민박', '숙소', '휴식', '안전인증민박', '바비큐', '농어촌민박', '해수욕장', '주방기구']
241 	 메이풀하우스 	 ['휴양콘도', '숙소', '펜션', '휴양펜션', '가족', '주차장', '단체여행객', '리조트']
247 	 느르왓 	 ['민박', '숙소', '고향민박', '안전인증민박', '주차장', '농어촌민박']
288 	

In [19]:
def minus_1(x):
  return x-1

In [20]:
accom.iloc[list(map(minus_1,check_list))]

,id,title,roadaddress,popular_tag
1336,39,틸다하우스,제주특별자치도 제주시 한경면 중산간서로 3367-1,"[숙소, 휴식, 조식, 바비큐, 해수욕장, 공공와이파이존]"
2627,54,소풍(월정리),제주특별자치도 제주시 구좌읍 월정1길 23,"[숙소, 안전인증민박, 조식 포함, 해수욕장, 자연경관]"
1504,64,이디하우스n카페,제주특별자치도 제주시 구좌읍 해녀박물관길 33-2,"[게스트하우스, 숙소, 휴식, 조식, 주차장, 카페, 자연경관, 독채, 공공와이파이존]"
3867,65,비안,제주특별자치도 제주시 조천읍 중산간동로 1197-16,"[게스트하우스, 숙소, 휴식, 주차장, 조식 포함, 독채, 공공와이파이존]"
3464,67,마이테르유스호스텔,제주특별자치도 제주시 애월읍 애원로 474-29,"[힐링쉼터, 수영장]"
...,...,...,...,...
660,917,씨스테이 용두암,제주특별자치도 제주시 용마로4길 15,[숙소]
2880,918,유창민박,제주특별자치도 제주시 추자면 추자로 46-1,"[민박, 숙소, 추자도, 주차장, 농어촌민박]"
633,919,더오라,제주특별자치도 제주시 동성길 36-1 3층,"[조식 포함, 숙소, 공공와이파이존]"
1787,920,보로미 민박,제주특별자치도 제주시 애월읍 고성서3길 20-1,[숙소]


In [26]:
accom.iloc[list(map(minus_1,check_list))].to_csv("/content/drive/MyDrive/yeoreodigm/data_files/09.19_accomodation_check_list.csv",encoding='euc-kr')

##travel_note title도 수정

In [ ]:
import psycopg2
from tqdm import tqdm #진척도 확인을 위한 Progress bar 관련 라이브러리

In [ ]:
def load_db():
  with open("/content/drive/MyDrive/yeoreodigm/data_files/db_info.json") as json_file:
    DB_INFO = json.load(json_file)
  
  endpoint = DB_INFO["ENDPOINT"]
  dbname = DB_INFO["DB_NAME"]
  user = DB_INFO["USER_ID"]
  password = DB_INFO["PASSWORD"]
  db = psycopg2.connect(host=endpoint,dbname=dbname,user=user,password=password)
  return db

In [ ]:
db=load_db()
cursor=db.cursor()

In [ ]:
title_dict= {0:"신나게 즐겨보는 Active in JEJU",1:"맑고 푸르른 제주의 쪽빛 바다",2:"오랜 세월이 빚어낸 푸르른 제주",3:"쉿! 우리 같이 걸어요 제주.",4:"실내에서도 즐기는 제주 인문학 탐방",\
             5:"부모님과 함께라면, 더 좋은 제주",6:"제주 감성 여행, 인생샷 건지기",7:"직접 몸으로 체험하는 제주의 자연",8:"제주, 다양하게 즐기기",9:"화산이 빚어낸 제주의 고요한 녹빛",\
             10: "실내외를 아우르는 제주 여행",11:"제주에선, 잠시 쉬어가도 괜찮아."}

In [ ]:
course = pd.read_csv("/content/drive/MyDrive/yeoreodigm/data_files/08.21_course.csv",index_col=0)

In [ ]:
for crs in course.values:
  
  id = crs[0]
  cluster = crs[3]
  title = title_dict[cluster]
  print(id,cluster,title)
  break

1 10 실내외를 아우르는 제주 여행


In [ ]:
db.rollback()

In [ ]:
for crs in tqdm(course.values):
  id = crs[0]
  cluster = crs[3]
  title = title_dict[cluster]
  sql_querry = f"update travel_note set title=\'{title}\' where travel_note_id={id}"
  cursor.execute(sql_querry)
db.commit()

100%|██████████| 3725/3725 [03:16<00:00, 18.94it/s]


##db적용

In [21]:
check_list = pd.read_csv("/content/drive/MyDrive/yeoreodigm/data_files/09.19_accomodation_check_list.csv",encoding='cp949')
check_list = check_list[['id','title','accom_type']]
check_list.head(2)

,id,title,accom_type
0,39.0,틸다하우스,펜션
1,54.0,소풍(월정리),게스트하우스


In [22]:
check_list.shape

(37624, 3)

In [23]:
check_list = check_list.iloc[:141]
check_list.shape

(141, 3)

In [26]:
check_list

,id,title,accom_type
0,39.0,틸다하우스,펜션
1,54.0,소풍(월정리),게스트하우스
2,64.0,이디하우스n카페,게스트하우스
3,65.0,비안,게스트하우스
4,67.0,마이테르유스호스텔,호텔/리조트
...,...,...,...
136,917.0,씨스테이 용두암,펜션
137,918.0,유창민박,민박
138,919.0,더오라,펜션
139,920.0,보로미 민박,민박


In [27]:
check_list['id'] = check_list['id'].apply(lambda x: int(x))

In [28]:
check_list

,id,title,accom_type
0,39,틸다하우스,펜션
1,54,소풍(월정리),게스트하우스
2,64,이디하우스n카페,게스트하우스
3,65,비안,게스트하우스
4,67,마이테르유스호스텔,호텔/리조트
...,...,...,...
136,917,씨스테이 용두암,펜션
137,918,유창민박,민박
138,919,더오라,펜션
139,920,보로미 민박,민박


In [30]:
type_list = []
for acc in accom.values:
  # print(acc)
  id,title,address, tag = acc[:]
  my_type = []
  
  
  #호텔/콘도/리조트, 민박, 펜션, 게스트하우스 이게 명칭으로 있는거는 확정짓기
  if '호텔' in title or '콘도' in title or '리조트' in title:
    type_list.append('호텔/콘도')
    continue
  elif '펜션' in title:
    type_list.append('펜션/민박')
    continue
  elif '게스트하우스' in title:
    type_list.append('게스트하우스')
    continue
  elif '모텔' in title:
    type_list.append('모텔')
    continue
  else:    
    for key in accom_type.keys():
      if set(accom_type[key]) & set(tag) != set(): # 교집합이 있다면
        # print(set(accom_type[key]) & set(tag))
        my_type.append(key)
    
  if len(my_type) != 1: #해당하는 타입이 하나도 없거나 중복된다면 체크리스트에 넣자
    type_list.append('')
  else:
    type_list.append(my_type[0])

In [31]:
len(type_list)

922

In [32]:
accom['accom_type'] = type_list

In [33]:
np.nan

nan

In [34]:
def reverToNull(x):
  if x=='':
    return np.nan
  else:
    return x

In [35]:
accom['accom_type'] =accom['accom_type'].apply(reverToNull)
accom['accom_type']

3378    게스트하우스
3801     호텔/콘도
3976        펜션
4240     호텔/콘도
3162     호텔/콘도
         ...  
2880       NaN
633        NaN
1787       NaN
2885       NaN
1175        펜션
Name: accom_type, Length: 922, dtype: object

In [36]:
accom['accom_type'].isnull().sum()

141

In [37]:
accom[accom['accom_type'].isnull()]

,id,title,roadaddress,popular_tag,accom_type
1336,39,틸다하우스,제주특별자치도 제주시 한경면 중산간서로 3367-1,"[숙소, 휴식, 조식, 바비큐, 해수욕장, 공공와이파이존]",NaN
2627,54,소풍(월정리),제주특별자치도 제주시 구좌읍 월정1길 23,"[숙소, 안전인증민박, 조식 포함, 해수욕장, 자연경관]",NaN
1504,64,이디하우스n카페,제주특별자치도 제주시 구좌읍 해녀박물관길 33-2,"[게스트하우스, 숙소, 휴식, 조식, 주차장, 카페, 자연경관, 독채, 공공와이파이존]",NaN
3867,65,비안,제주특별자치도 제주시 조천읍 중산간동로 1197-16,"[게스트하우스, 숙소, 휴식, 주차장, 조식 포함, 독채, 공공와이파이존]",NaN
3464,67,마이테르유스호스텔,제주특별자치도 제주시 애월읍 애원로 474-29,"[힐링쉼터, 수영장]",NaN
...,...,...,...,...,...
660,917,씨스테이 용두암,제주특별자치도 제주시 용마로4길 15,[숙소],NaN
2880,918,유창민박,제주특별자치도 제주시 추자면 추자로 46-1,"[민박, 숙소, 추자도, 주차장, 농어촌민박]",NaN
633,919,더오라,제주특별자치도 제주시 동성길 36-1 3층,"[조식 포함, 숙소, 공공와이파이존]",NaN
1787,920,보로미 민박,제주특별자치도 제주시 애월읍 고성서3길 20-1,[숙소],NaN


In [38]:
accom = accom[['id','title','roadaddress','popular_tag','accom_type']]
accom

,id,title,roadaddress,popular_tag,accom_type
3378,1,제주조천스위스마을게스트하우스,제주특별자치도 제주시 조천읍 함와로 566-27,"[게스트하우스, 숙소]",게스트하우스
3801,2,해비치호텔앤드리조트,제주특별자치도 서귀포시 표선면 민속해안로 537,"[호텔, 숙소, 양식레스토랑, 가족호텔, 주차장, 수영장, 리조트, 공공와이파이존,...",호텔/콘도
3976,3,나미송 머무는 곳,제주특별자치도 제주시 애월읍 평화로 2476,"[민박, 고향민박, 정원, 주방기구, 자연경관, 독채]",펜션
4240,4,신라호텔제주,제주도 서귀포시 중문관광로 72번길 75,"[호텔, 게스트하우스, 숙소, 양식레스토랑, 주차장, 수영장, 공공와이파이존, 관광호텔]",호텔/콘도
3162,5,롯데호텔 제주,제주특별자치도 서귀포시 중문관광로72번길 35,"[호텔, 숙소, 조식, 양식레스토랑, 가족호텔, 주차장, 자연경관, 수영장, 리조트...",호텔/콘도
...,...,...,...,...,...
2880,918,유창민박,제주특별자치도 제주시 추자면 추자로 46-1,"[민박, 숙소, 추자도, 주차장, 농어촌민박]",NaN
633,919,더오라,제주특별자치도 제주시 동성길 36-1 3층,"[조식 포함, 숙소, 공공와이파이존]",NaN
1787,920,보로미 민박,제주특별자치도 제주시 애월읍 고성서3길 20-1,[숙소],NaN
2885,921,밀물민박,제주특별자치도 제주시 추자면 추자로 106-1,"[민박, 숙소, 힐링, 추자도, 농어촌민박, 공공와이파이존]",NaN


In [39]:
join_type = accom.merge(check_list,on='id',how='left')

In [40]:
join_type

,id,title_x,roadaddress,popular_tag,accom_type_x,title_y,accom_type_y
0,1,제주조천스위스마을게스트하우스,제주특별자치도 제주시 조천읍 함와로 566-27,"[게스트하우스, 숙소]",게스트하우스,NaN,NaN
1,2,해비치호텔앤드리조트,제주특별자치도 서귀포시 표선면 민속해안로 537,"[호텔, 숙소, 양식레스토랑, 가족호텔, 주차장, 수영장, 리조트, 공공와이파이존,...",호텔/콘도,NaN,NaN
2,3,나미송 머무는 곳,제주특별자치도 제주시 애월읍 평화로 2476,"[민박, 고향민박, 정원, 주방기구, 자연경관, 독채]",펜션,NaN,NaN
3,4,신라호텔제주,제주도 서귀포시 중문관광로 72번길 75,"[호텔, 게스트하우스, 숙소, 양식레스토랑, 주차장, 수영장, 공공와이파이존, 관광호텔]",호텔/콘도,NaN,NaN
4,5,롯데호텔 제주,제주특별자치도 서귀포시 중문관광로72번길 35,"[호텔, 숙소, 조식, 양식레스토랑, 가족호텔, 주차장, 자연경관, 수영장, 리조트...",호텔/콘도,NaN,NaN
...,...,...,...,...,...,...,...
917,918,유창민박,제주특별자치도 제주시 추자면 추자로 46-1,"[민박, 숙소, 추자도, 주차장, 농어촌민박]",NaN,유창민박,민박
918,919,더오라,제주특별자치도 제주시 동성길 36-1 3층,"[조식 포함, 숙소, 공공와이파이존]",NaN,더오라,펜션
919,920,보로미 민박,제주특별자치도 제주시 애월읍 고성서3길 20-1,[숙소],NaN,보로미 민박,민박
920,921,밀물민박,제주특별자치도 제주시 추자면 추자로 106-1,"[민박, 숙소, 힐링, 추자도, 농어촌민박, 공공와이파이존]",NaN,밀물민박,민박


In [51]:
join_type.isnull()

,id,title_x,roadaddress,popular_tag,accom_type_x,title_y,accom_type_y
0,False,False,False,False,False,True,True
1,False,False,False,False,False,True,True
2,False,False,False,False,False,True,True
3,False,False,False,False,False,True,True
4,False,False,False,False,False,True,True
...,...,...,...,...,...,...,...
917,False,False,False,False,True,False,False
918,False,False,False,False,True,False,False
919,False,False,False,False,True,False,False
920,False,False,False,False,True,False,False


In [54]:
result = []
for val in join_type.values:
  type_x = val[-3]
  type_y = val[-1]
  
  if(type(type_x) == float):
    result.append(type_y)
  elif type(type_y) == float:
    result.append(type_x)

In [56]:
join_type['type'] = result

In [58]:
join_type = join_type[['id','title_x','roadaddress','popular_tag','type']]

In [60]:
join_type.isnull().sum()

id             0
title_x        0
roadaddress    0
popular_tag    0
type           0
dtype: int64

In [61]:
join_type.head(3)

,id,title_x,roadaddress,popular_tag,type
0,1,제주조천스위스마을게스트하우스,제주특별자치도 제주시 조천읍 함와로 566-27,"[게스트하우스, 숙소]",게스트하우스
1,2,해비치호텔앤드리조트,제주특별자치도 서귀포시 표선면 민속해안로 537,"[호텔, 숙소, 양식레스토랑, 가족호텔, 주차장, 수영장, 리조트, 공공와이파이존,...",호텔/콘도
2,3,나미송 머무는 곳,제주특별자치도 제주시 애월읍 평화로 2476,"[민박, 고향민박, 정원, 주방기구, 자연경관, 독채]",펜션


In [62]:
for accom in join_type.values:
  id,title,type_ = accom[0],accom[1],accom[-1]
  print(id,title,type_)

1 제주조천스위스마을게스트하우스 게스트하우스
2 해비치호텔앤드리조트 호텔/콘도
3 나미송 머무는 곳 펜션
4 신라호텔제주 호텔/콘도
5 롯데호텔 제주 호텔/콘도
6 대명리조트 제주 호텔/콘도
7 휘닉스 제주 섭지코지 호텔/콘도
8 캠핑트리펜션 펜션/민박
9 제주가좋아서 펜션 펜션/민박
10 금호리조트제주 호텔/콘도
11 제주도푸른바다 펜션
12 씨에스호텔앤리조트 호텔/콘도
13 마레보 리조트 호텔/콘도
14 르페도라 펜션
15 메종글래드 제주 호텔/콘도
16 한화리조트 제주 호텔/콘도
17 야호비치하우스 펜션
18 라마다프라자제주호텔 호텔/콘도
19 inn jeju 인제주 게스트하우스 게스트하우스
20 객의하우스 게스트하우스
21 서귀포KAL호텔 호텔/콘도
22 수상한소금밭게스트하우스 게스트하우스
23 신라스테이 제주 호텔/콘도
24 포도호텔 호텔/콘도
25 제주bp게스트하우스 JEJU BP GUESTHOUSE 게스트하우스
26 아이비테라스 펜션
27 비젠빌리지 펜션
28 오션스위츠호텔 제주 호텔/콘도
29 메종드오조락 게스트하우스
30 히든클리프 호텔 앤 네이쳐 호텔/콘도
31 베이힐 풀앤빌라 호텔/콘도
32 랜딩관 제주신화월드 호텔앤리조트 호텔/콘도
33 롯데리조트아트빌라스 호텔/콘도
34 아름다운리조트 호텔/콘도
35 제주도푸른산푸른바다 게스트하우스
36 베스트웨스턴제주호텔 호텔/콘도
37 켄싱턴리조트 제주 한림점 호텔/콘도
38 플레이스 캠프 제주 호텔/콘도
39 틸다하우스 펜션
40 골든튤립 제주 성산호텔 호텔/콘도
41 대명 샤인빌리조트 호텔/콘도
42 돈내코힐 리조트 호텔/콘도
43 봄그리고가을호텔&리조트 호텔/콘도
44 롯데시티호텔제주 호텔/콘도
45 나날 게스트하우스 게스트하우스
46 더본 호텔 호텔/콘도
47 제주아이브리조트 호텔/콘도
48 우도피아 펜션
49 Oh! 세화! (오세화게스트하우스) 게스트하우스
50 더클라우드호텔 호텔/콘도
51 더포그레이스호텔앤리조트 호텔/콘도
52 제주 에코 스위츠 펜션 펜션/민박
53 게으른소나기 게스트하우

##DB에 숙소타입 등록

In [63]:
from tqdm import tqdm
import psycopg2
def load_db():
  with open("/content/drive/MyDrive/yeoreodigm/data_files/db_info.json") as json_file:
    DB_INFO = json.load(json_file)
  
  endpoint = DB_INFO["ENDPOINT"]
  dbname = DB_INFO["DB_NAME"]
  user = DB_INFO["USER_ID"]
  password = DB_INFO["PASSWORD"]
  db = psycopg2.connect(host=endpoint,dbname=dbname,user=user,password=password)
  return db

In [64]:
db = load_db()
cursor=db.cursor()

In [76]:
db.rollback()

In [77]:
for accom in tqdm(join_type.values):
  id,title,type_ = accom[0],accom[1],accom[-1]
  sql = f"update accomodation set type = \'{type_}\' where accomodation_id = {id}"
  cursor.execute(sql)
db.commit()

100%|██████████| 922/922 [00:48<00:00, 19.10it/s]
